In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/Github/Abstract-generator/notebooks

In [3]:
%%capture
if IN_COLAB:
  !pip install feedparser tokenizers transformers scipy==1.7.1;

In [4]:
import os, torch, time, math, sys, re, csv
import numpy as np

PACKAGE_ROOT = os.path.dirname(os.path.abspath(''))
print(PACKAGE_ROOT)
sys.path.append(PACKAGE_ROOT)

from src import settings
import src.data.dataset_class as dsc
import src.data.dataloader_class as dlc

from src.model.transformer_torch import TransformerModel
from src.model.generate_text import gen_some_text

from src.model.train_evaluate import train_version_jeremy as train
from src.model.train_evaluate import evaluate_version_jeremy as evaluate

#from src.model.transformer import make_gpt_model # imports don't work

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jbrothschild/Documents/HLML/Abstract-generator


### Parameters

In [5]:
# ARCHITECTURE
# TODO : Make a class that sets all this, validates it's in use
max_len_sentence     = 40 # maximum sentence length
vocab_size  = None # None if you want to let tokenizer do its thing
emsize     = 512 # embedding dimension
nhid       = 2048 # the dimension of the feedforward network model in torch.nn.TransformerEncoder
nlayers    = 12 # the number of torch.nn.TransformerEncoderLayer in torch.nn.TransformerEncoder
nhead      = 8 # the number of heads in the multiheadattention models
dropout    = 0.2 # the dropout value
batch_size = 10 #32
val_batch_size = 10 #32, not used right now.
epochs     = 50  # The number of epochs

TRAIN = True

### Format Dataset

Uses a custom dataset class, which is an iterable and callable structure that returns a sample from our dataset. Within this custom dataset, can determine all preprocessing.

In [6]:
# create dataset
dataset = dsc.ArxivDataset()
#dataset = dsc.WikiTextDataset()

#train tokenizer (or use one already trained)
tknzr_type = 'BPE'
flag_tknzr_train = True
flag_tknzr_fast = True

_ = dataset.tokenizer(flag_tknzr_train, tknzr_type, flag_tknzr_fast=flag_tknzr_fast)

### Creating DataLoaders

Training is done on batches, so we need a way to extract groupings of the data in the appropriate format for our transformer model.
Note that for transformers which we are training, dataloaders outputs both src (x[:-1] and tgt ([1:]).
The collation of batches for different transformer models we have vary. For HuggingFace it's ( max_len x batch_size ) whereas I think that the Annotated Transformer has ( batch_size x max_len ).

I created a custom Dataloader class that wraps splitting the dataset and also outputs different dataloaders for each.

NOTE : Do not use the tokenizer before the training if you use num_workers>0!
FastTokenizer does not play nicely with forking if you use it before the forking of your data:
https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning

In [7]:
flag_padding_mask = True

dataloader = dlc.CustomDataloader(dataset, batch_size, max_len_sentence, flag_padding_mask=flag_padding_mask) 

### Selecting model

Here we choose which model we shall use for training. For now, I've selected the black box Transformer from HuggingFace because the collate_fn I've written gives the correct input size force it... however this can easily be changed! 

In [8]:
# transformer from huggingface
# TODO : Change to the Annotated Transformer if I want
model = TransformerModel(dataset.vocab_size, emsize, nhead, nhid, nlayers, dropout).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()#ignore_index=tknzr.get_vocab()["<pad>"])

# optimizer TODO : why these parameters?
paramsAdam  = [{'params' : model.parameters(), 'lr' : 1e-3, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsAdamW = [{'params' : model.parameters(), 'lr' : 5e-5, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsSGD   = [{'params' : model.parameters(), 'lr' : 0.5, 'momentum' : 0.0, 'dampening' : 0.0, 'weight_decay' : 0.0}]

optimizer = torch.optim.SGD( paramsSGD )
#optimizer = torch.optim.Adam( paramsAdam )
#optimizer = torch.optim.AdamW( paramsAdamW )

# scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95) # 1.0 to signify no decay rate

### Training

Training loop!

In [9]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # fasttokenizer should not be used before forking. Something
                                                # to figure out. What this does is suppress some warning messages 
                                                # https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning
                                                # doesn't seem to affect the timing though
if TRAIN:
    best_val_loss = float("inf")
    best_model = None
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model, dataloader.train, device, dataset.vocab_size, epoch, optimizer, scheduler, criterion, max_len_sentence)
        val_loss = evaluate(model, dataloader.valid, device, dataset.vocab_size, criterion, max_len_sentence)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
              'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                         val_loss, math.exp(val_loss)))
                                         # Why is math.exp so large????
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

        scheduler.step()

    # save best model (two methods)
    model_full = settings.DIR_MODELS + os.sep + f'{dataset.name}_epoch{epochs}.pth'
    model_weights = settings.DIR_MODELS + os.sep + f'{dataset.name}_weights_epoch{epochs}.pth'
    model_full_best = settings.DIR_MODELS + os.sep + f'{dataset.name}_epoch{epochs}_best.pth'
    model_weights_best = settings.DIR_MODELS + os.sep + f'{dataset.name}_weights_epoch{epochs}_best.pth'
    # approach 1: save model (class) entirely (uses pickle)
    torch.save(model, model_full)
    torch.save(best_model, model_full_best)
    # approach 2: save model weights
    torch.save(best_model.state_dict(), model_weights_best)

| epoch   1 |   200/  700 batches | lr 0.50 | ms/batch 959.29 | loss  7.82 | ppl  2500.78
| epoch   1 |   400/  700 batches | lr 0.50 | ms/batch 971.15 | loss  7.07 | ppl  1172.85
| epoch   1 |   600/  700 batches | lr 0.50 | ms/batch 967.55 | loss  6.90 | ppl   993.80
39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 3098,   213,   213,  3061,  2987,   189,   213,   213,   213,   189],
        [  146,  1383,  1091,   130,   130,  1741,   566,  1930,   714,  1662],
        [  374,   121,   514,   533,   692,  1479,   105,   146,   107,  1902],
        [  202,   133,   130,  4550,   592,   162,   613,  3446,   611,   122],
        [  109,  4251,   904,   187,   202,   107,   311,  2879,   122,   508],
        [  130,  1313,  1039,   416,  9425,  1197, 22456,   171,  1159,   130],
        [  138,  1508,   122,   280,   205,  6280,   909,   107,  2783,  2957],
        [  205,  8243,   280,   130,  1172,   122,   130,  4073,   175,  2044],
       

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [16159,   920,   189, 13041,   189,  1089,   213,   189,   339,   213],
        [ 3676,  3676,  1192,   122,   247,   130,  1013,   957,  1389,  1042],
        [  122,   786,  2389,   105,   420,   663,   107,   420,  1032,  5007],
        [ 2595,  2062,   171,  3497,   121,  2484,  4113,   391,   143,   541],
        [  665,   151,   105,  7010,  3181,   162,  9604,   122,   246,   122],
        [ 1690,   107,  6365,   130,  6078,   105,   122,  1486,  1181, 11329],
        [  146,   134,   476,  5071,  3173,  1798,   134,   130,   107,   210],
        [  415,  2342, 10520,  4240,   187,  3441,   130,   134,  2315,   121],
        [  130,  2237,   151,   391,  4579,   121,  1800, 12124,   409,  4396],
        [  555,   143,  6191,  3063,   391,  2642,   467,  1291,   210,  2318],
        [  134,   107,   105,   900,   162,   199,   121,   334,   146,   166],
        [ 1362,  2947, 17348,  4726, 

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  339,   213, 14679,   189,  5246,   213,   213, 19667,   189,  2284],
        [  107,  1672,  4379,   957,  2276,  1013,   553,   130,  3053,   107],
        [ 1659,   146,  4898,   130,   162,   107,   107, 19109,  1400,  1729],
        [ 1033,  1730,  2032,   420,   133,   431,  4119,  1705,   146,   122],
        [  863,   979,   295,   134, 10767,   356,   122,  4994,  3053,   266],
        [ 5665,   105,   723,   383,   266,   374,   107,   202,   102,   723],
        [ 1129,   748,   122,   122,   130,   307,  3424, 21781,  1369,   246],
        [ 3922,  4113,   107,   105,  6660,   879,  7977,  1001,  1251,  1013],
        [  508,   613,   266,   469,   662,   122,  1404,  5316,  1273,   107],
        [  134,   143,  3849,   130,   121,   107,   412,  2480,   211,  3898],
        [  597,  3158,   151,   533,  4934,  2086,    67,  3264,   910,   409],
        [  787,   130,   555,   280, 

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  213, 15920,   213,   249, 19441,  3061,  5425,   213,   189,   213],
        [  876,   130,   510,   979,   162,   130,   122,  1930,   631,  2548],
        [  105,  5509,  1269,   105,   107,   134,   508,  2879,   671,   107],
        [ 4162,  1419,   383,  8499,  1405,   483,   130,   171,   202,  1450],
        [ 1218,   130,   130,   311,  3542,  1049,  7486,   107,  4212,  5139],
        [  542,  1319,  1053,  3618,   146,   268,  2044, 16966,   205,  3064],
        [  171,  1742,   542,  1686, 11295,   146,  3000,  2551,   122,   122],
        [  134,  1162,   202,   246,  2972,   293,   187,   130,   133,   508],
        [  280,   202,  2405,  5219,   121,  1049,   134,   474,  1935,   134],
        [  997,  2697,   205,   199,   107,   268,  3081,  1106,   166,  1033],
        [  121,   205,   683,   107,  2074,  1339,   146,   151,   125,   787],
        [ 2439,   574,   723,   631, 

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  213,  1334,   213,  8662,   189,   213,  2993,   213,  6689,   213],
        [ 1013,   107,   510,   900,  4139,  1396,   105,   566,  2623,   335],
        [  107,  3826,   778,  5887,   959,   107,   945,   107,  1362,   199],
        [  356,   130,   431,   510,   122,   706,   130,   451,   510,   107],
        [ 1463,  2347,   356,  1754,  8582,   656,   307,   122,   133,  5213],
        [  122,   130,  2124,  7530,  5748,   398,  1545,   786,  5414, 10159],
        [  107,  1789,  1033,   225,   202,   143,   122,   130,  8160,  2310],
        [  247,   925,  1162,  2812,  4002,   166,   107,  1213,   121,  4597],
        [  391,   143,   146,   322,   205,   186,  1400,  1385,  1304,  7290],
        [  122,   246,  1105,   130,   210,   202,   130,  1039,   122,   208],
        [ 1806,   553,  3740,   533,   146,   370,  3053,   122,   641,   105],
        [17062,   107,   151,   941, 

...

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [20258,   213,   189,   213,  4375,  5199,  8436,   189,  1082, 24789],
        [  208,   977,  4458,  2983,   143,   130,  1319,  2102,  1747,   134],
        [ 1389,   799,   122,   107,   105,  1319,  1742,  1244,   122,  1495],
        [ 5559,  3093,  1033,   209,  6443,  2023,  1162,   122,   107,   121],
        [ 1815,   130,   322,   130,  2646,   856,   202,   107,   134,  2439],
        [  122,  5388,   130,  1719,   122,   130,  2697,  7866,   130,  2758],
        [ 2116,  1162,   692,   134,  1253,   986,   205,   900,   423,  8901],
        [ 3104,   151,   592,   130,   574,   474,  2421,  2702,   295,   334],
        [  130,   561,   202,   134,   643,   288,   107,  2695,   175,  8747],
        [ 1144,   280,  9425,   768,   689,   884,  1444,   130,  1164,  1991],
        [ 2529,  2026,   205,   151,   121,   130,   122, 11514,   811,   122],
        [  967,   122,   171,   107, 

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  213,   189,   516,   213,   189,   339,   213,   213,   213,   256],
        [  566,  3023,  3619,  2936,  1771,   343,   714,  1930,   566,   665],
        [  105,   122,   162,   105,   585,  1313,   105,   133,   105,   130],
        [ 1048,   134,  8088,  1107,   122,   246,   351,   826,   553,  1422],
        [ 2619,   897,   151,  1601,   107,   510,  1590,   247,   122,  5040],
        [ 4060,   661,   107,   151,   134,   910,   171,  1590,   107,   745],
        [  171,   134,  2782,  2875,   861,   107,   105,   171,   247,   374],
        [  107,  2003,   122,  1178,   175,   368,  4247,  3235,   391,   307],
        [ 1545,   216,   107,   356,   105,   122,  1366,   806,   143,   162],
        [  122,  1169,  2838,   247,   322,  1556,   620,  1400,   423,  1681],
        [  107,   431,  6426,   541,   130,   374,   107,   130,  2282,   151],
        [  620,   266,   121,   208, 

39 tensor([[    1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  213,  1465,  6100,   213,   256, 10280,   189, 16015,   256,   213],
        [ 1042,  7360, 26079,   553,  1385,  2023,  1089,   107,   803,  1960],
        [  133,   122,   202,   107,  2600,   856, 13751,   134,   122,   107],
        [ 2023,   671,  7027,   541,   171,   130,  6032,  2052,  2797,   368],
        [  134,  3780,   205,   122, 12071,   986,   202,   886,   541,   122],
        [ 2016,   208,   162,  1666,  2804,  1135,  8779,   208,   122,   134],
        [  553,  9068,   105,  3780,  2120,  2051,   205,   134,   770,   130],
        [  122,   210,  1659,   121,   134,  3560,  1369,  1778,   146,   423],
        [ 7669,   574,   134,   322,  4138,  3158,   225,   121,  4919,   467],
        [ 5331,   823,  4699,   130,   121,  2023,  1383,  1745,   211,   175],
        [  661,   643,   143,  2057,   134,  1314,  7521,  6963, 13641,  1742],
        [  671,  5477,  2826,   134, 

-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 692.74s | valid loss  6.82 | valid ppl   916.16
-----------------------------------------------------------------------------------------


KeyboardInterrupt: 

### Text Generation

Here I've simply taken the code Matt uses to generate text.

In [ ]:
if not TRAIN:
    custom_filename = 'arxiv_10000'
    custom_epochs = 10
    model_full = settings.DIR_MODELS + os.sep + f'{custom_filename}_epoch{custom_epochs}_best.pth'
    model_weights = settings.DIR_MODELS + os.sep + f'{custom_filename}_weights_epoch{custom_epochs}_best.pth'
    
    # approach 1: load model (class) entirely (uses pickle)
    model_full_load = torch.load(model_full, map_location=device)

    # approach 2: load model weights, need to have some parameter or something 
    model_load = TransformerModel(vocab_size, emsize, nhead, nhid, nlayers, dropout).to(device)
    model_weights_load = model_load.load_state_dict( torch.load(model_weights) )

In [ ]:
# inspect both models
#print('model_A info...\n', model_full_load)
#print('\nmodel_B info...\n', model_weights_load)

#print('model_A == model_B:', model_full_load == model_weights_load)
#model = model_full_load
# Text generation example

#model = model_load
prompt = 'The dog ran'
ngen = 100
decode_style = 'sample_topp' #greedy, sample_topp
model.to('cpu')
generated_text = gen_some_text(
    best_model, dataset.transform, 'cpu', max_len_sentence, text_prompt=prompt, tokens_to_gen=ngen, vis=False,
    decode_style=decode_style)
print("Text prompt:\n", prompt)
print("Number of tokens to generate:", ngen)
print("Generated_text:\n", generated_text)

# TODO: alternative generation
# currently 'greedy method'
# see: https://huggingface.co/blog/how-to-generate